In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model_path = '/home/uework/AiWeb/pwz/github/model/Nous-Hermes-Llama2-13b'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype='auto',quantization_config=quantization_config)

Loading checkpoint shards: 100%|██████████| 3/3 [00:37<00:00, 12.34s/it]


In [7]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

print(list_files('/home/uework/AiWeb/pwz/github/luacoder/data'))

data/
    json.ipynb
    lua.ipynb
    utils.py
    dialogue.ipynb
    requirements.txt
    README.md
    scripts/
        arguments.py
        bpe_training.py
        codeparrot_training.py
        human_eval.py
        initialize_model.py
        minhash_deduplication.py
        preprocessing.py
        pretokenizing.py
        validation_loss.py
        preprocess.sh
        tests/
            test_deduplicate.py
            __init__.py
        __pycache__/
            arguments.cpython-310.pyc
            minhash_deduplication.cpython-310.pyc
            preprocessing.cpython-310.pyc
    __pycache__/
        utils.cpython-310.pyc
None


In [9]:

message = "Hello, my dog is cute"
input_ids = tokenizer(message, return_tensors="pt").input_ids.to("cuda")
generated_ids = model.generate(input_ids, max_new_tokens=250, do_sample=True, repetition_penalty=1.3, temperature=0.8, top_p=0.75, top_k=40)
response = tokenizer.decode(generated_ids[0][input_ids.shape[-1]:])
print(response)

2023-09-07 03:59:41.346131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 03:59:42.003505: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


!
I have a cat named Snoop. She's so adorable and fluffy that I just had to share her with the world.</s>


In [8]:
prompt = '--查看怪物id'
inputs = tokenizer(prompt, return_tensors="pt").to('cuda:0')

import time
start_time = time.time()
with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False):
    outputs = model.generate(**inputs,)
    # outputs = model.generate(**inputs, max_length=50, num_return_sequences=1,do_sample=True, temperature=1.0, top_k=50, top_p=0.95,pad_token_id=tokenizer.eos_token_id,eos_token_id=49155)
# Stop the timer
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time


print(f'Generate token: {len(outputs[0])/elapsed_time:.2f} token/s')
print(tokenizer.decode(outputs[0]))

ValueError: The following `model_kwargs` are not used by the model: ['token_type_ids'] (note: typos in the generate arguments will also show up in this list)

In [1]:
from vllm import LLM, SamplingParams
prompts = [
    '--查看怪物id',
    "--学习技能",
    "--学习技能并查看怪物id",
    "--学习技能并查看怪物id并打印出来",
]
sampling_params = SamplingParams(temperature=0.2, top_p=0.95, top_k=4, max_tokens=128)

/home/uework/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-21 05:16:15,906	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-09-21 05:16:16,286	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [1]:
from vllm import LLM, SamplingParams
llm = LLM(model="/home/uework/AiWeb/pwz/github/model/luacoder-3b/beta2/checkpoint-215")

INFO 09-21 07:16:23 llm_engine.py:72] Initializing an LLM engine with config: model='/home/uework/AiWeb/pwz/github/model/luacoder-3b/beta2/checkpoint-215', tokenizer='/home/uework/AiWeb/pwz/github/model/luacoder-3b/beta2/checkpoint-215', tokenizer_mode=auto, trust_remote_code=False, dtype=torch.bfloat16, download_dir=None, load_format=auto, tensor_parallel_size=1, seed=0)
INFO 09-21 07:16:31 llm_engine.py:199] # GPU blocks: 105648, # CPU blocks: 14563


In [3]:
import time
start_time = time.time()
outputs = llm.generate(prompts, sampling_params,use_tqdm=True)
# Stop the timer
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

token_num = 0
print(outputs)
# Print the outputs.
for output in outputs:
    token_num += len(output.outputs[0].token_ids)
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print('----------------------------------------')
    print(f"Prompt: {prompt!r}\nGenerated text:")
    print(generated_text)
print(f'Generate token: {token_num/elapsed_time:.2f} token/s')

Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  2.01it/s]

[RequestOutput(request_id=0, prompt='--查看怪物id', prompt_token_ids=[287, 24044, 3591, 122, 14615, 314], outputs=[CompletionOutput(index=0, text='\nfunction isc_test_creature:get_id()\n    local creature = edi.player:owner(module.player)\n    local id = edi.creature:get_id(creature)\n    LOGI("isc_test_creature get_id: " .. id)\nend\n\n--查看怪物名字\nfunction isc_test_creature:get_name()\n    local creature = edi.player:owner(module.player)\n    local name = edi.creature:get_name(creature)\n    LOGI("isc_test_creature get_name: " ..', token_ids=[203, 1126, 438, 85, 81, 858, 81, 42717, 44, 371, 81, 314, 346, 284, 2196, 29472, 280, 2115, 91, 32, 3840, 44, 5632, 26, 1647, 32, 3840, 27, 284, 2196, 804, 280, 2115, 91, 32, 42717, 44, 371, 81, 314, 26, 42717, 27, 284, 5776, 59, 440, 25415, 81, 858, 81, 42717, 622, 81, 314, 44, 313, 4377, 804, 27, 203, 416, 203, 203, 287, 24044, 3591, 122, 14615, 38351, 203, 1126, 438, 85, 81, 858, 81, 42717, 44, 371, 81, 426, 346, 284, 2196, 29472, 280, 2115, 91, 32,

In [8]:
import os

import os

def list_files(startpath, skip_dirs=[]):
    for root, dirs, files in os.walk(startpath):
        # Remove the skipped directories from the list of directories to walk through
        dirs[:] = [d for d in dirs if d not in skip_dirs]
        
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

list_files('/home/uework/AiWeb/pwz/github/luacoder/data', skip_dirs=["node_modules", "__pycache__","wandb","logs"])


data/
    json.ipynb
    lua.ipynb
    utils.py
    dialogue.ipynb
    requirements.txt
    README.md
    scripts/
        arguments.py
        bpe_training.py
        codeparrot_training.py
        human_eval.py
        initialize_model.py
        minhash_deduplication.py
        preprocessing.py
        pretokenizing.py
        validation_loss.py
        preprocess.sh
        tests/
            test_deduplicate.py
            __init__.py
